In [8]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
#弃用mnist集
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
      serialized_example,
      features={
          
          'image_data': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
      })

    image = tf.decode_raw(features['image_data'], tf.uint8)
    image = tf.reshape(image, [28, 28,3])

    #image = tf.cast(image, tf.float32) * (1. / 255)
    image = tf.cast(image, tf.float32) 

    label = tf.cast(features['label'], tf.int32)

    return image, label


def inputs(filename, batch_size):
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
          [filename], num_epochs=5000)

        image, label = read_and_decode(filename_queue)

        images, labels = tf.train.shuffle_batch(
          [image, label], batch_size=batch_size, num_threads=1,
          capacity=400,min_after_dequeue=200)

    return images,labels

learning_rate = 1e-4
keep_prob_rate = 0.7 # 
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, w):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# define placeholder for inputs to network
##数据读
batch_xs, batch_y =inputs("./train.tfrecords",100)
#标签层转换
batch_ys=tf.one_hot(batch_y, 10, on_value=1.0, off_value=0.0)
#xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = keep_prob_rate
x_image = tf.reshape(batch_xs, [-1, 28, 28, 3])


#  卷积层 1
## conv1 layer ##

W_conv1 =  weight_variable([7, 7, 3, 32])                      # patch 7x7, in size 1, out size 32
b_conv1 =  bias_variable([32])                    
h_conv1 =  tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)                    # 卷积  自己选择 选择激活函数
h_pool1 =  max_pool_2x2(h_conv1)                     # 池化               

# 卷积层 2
W_conv2 =  weight_variable([5, 5, 32, 64])                      # patch 5x5, in size 32, out size 64
b_conv2 =  bias_variable([64])
h_conv2 =  tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)                      # 卷积  自己选择 选择激活函数
h_pool2 =  max_pool_2x2(h_conv2)                      # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(batch_ys* tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

saver=tf.train.Saver(max_to_keep=1)
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    init2=tf.local_variables_initializer()
    sess.run([init,init2])
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
#这部分都是魔改 
#    try:
#        while not coord.should_stop():
    
    for i in range(40):    
        sess.run(train_step)
        print(compute_accuracy(
                batch_xs,batch_ys ))
    saver.save(sess,"./model.ckpt",global_step=1)
#    except tf.errors.OutOfRangeError:
#          pass
    coord.request_stop()

    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 2352 values, but the requested shape has 784
	 [[Node: Reshape_3 = Reshape[T=DT_UINT8, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](DecodeRaw_1, Reshape_3/shape)]]


OutOfRangeError: RandomShuffleQueue '_93_input_3/shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 1)
	 [[Node: input_3/shuffle_batch = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_3/shuffle_batch/random_shuffle_queue, input_3/shuffle_batch/n)]]

Caused by op 'input_3/shuffle_batch', defined at:
  File "/root/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda3/envs/tensorfl